In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from emmo.models.prediction import PredictorMHC2
from emmo.models.cleavage import CleavageModel
from emmo.utils.viz import plot_cleavage_model

In [ ]:
model_path = Path().absolute().parent / 'models'
model_path.mkdir(parents=True, exist_ok=True)

## Cleavage model

In [ ]:
cleavage_n_terminus = Path('/mnt/bfx/user_folders/david.schaller/PWM_EM/mhc2/cleavage/N-terminus/clusters3')
cleavage_c_terminus = Path('/mnt/bfx/user_folders/david.schaller/PWM_EM/mhc2/cleavage/C-terminus/clusters3')

cleavage_model_path = model_path / 'cleavage' / 'cleavage_MHCII_3N_3C_20230310'

cleavage_model = CleavageModel.load_from_separate_models(
    cleavage_n_terminus,
    cleavage_c_terminus,
)

# check if saving and reloading with method load() works
cleavage_model.save(cleavage_model_path)
cleavage_model = CleavageModel.load(cleavage_model_path)

In [ ]:
plot_cleavage_model(cleavage_model)

## Load the models for the alleles

In [ ]:
peptides_path = Path('/mnt/bfx/user_folders/david.schaller/PWM_EM/mhc2/sorted_peptides/')
deconvolution_results_path = Path('/mnt/bfx/user_folders/david.schaller/PWM_EM/mhc2/models_EMMo/')

In [ ]:
# maps allele to (model path, k, i) where
# k determines which subfolder to use (run with k classes) and
# i determines which motif to use from this subfolder (motif i)
selected_models = {}

for folder in deconvolution_results_path.iterdir():
    if folder.is_dir():
        allele = folder.stem
        selected_models[allele] = (folder, 1, 1)

# manually correct the motifs that seem to be off by visual inspection
selected_models['DRB11501-DRA0101'] = (selected_models['DRB11501-DRA0101'][0], 2, 2)
selected_models['DRB11502-DRA0101'] = (selected_models['DRB11502-DRA0101'][0], 2, 1)

print(len(selected_models))

In [ ]:
for output_dir, recompute in zip(['mhc2_msdb_2020_nests_train_tune_ppm_direct_20230522', 
                                  'mhc2_msdb_2020_nests_train_tune_ppm_recomputed_20230522'],
                                 [False, True]):

    predictor = PredictorMHC2.compile_from_selected_models(
        selected_models,
        cleavage_model_path,
        peptides_path,
        motif_length=9,
        length_distribution='MHC2_biondeep',
        recompute_PPMs_from_best_responsibility=recompute,  # <-- enable/disable PPM recomputation
    )

    available_alleles = set(predictor.PPMs.keys())

    print('Available for scoring:', len(available_alleles))

    predictor.save(model_path / 'binding-predictor' / output_dir, force=True)

### Plot the offset weights

In [ ]:
offset_weights = predictor.offset_weights

offset_weights /= np.sum(offset_weights)

max_offset = len(offset_weights) // 2
x = np.arange(-max_offset, max_offset + 1)

plt.bar(x, offset_weights)